In [20]:
import numpy as np
import pandas as pd

In [21]:
df = pd.read_csv('Churn_Modelling.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [22]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [23]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
X = scaler.fit_transform(X)

In [26]:
X.shape

(768, 8)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=1)

In [81]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [ ]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train, batch_size=32,epochs=100,validation_data=(X_test,y_test))

# 1. How to select appropriate optimizer_

In [29]:
import kerastuner as kt

In [30]:
def build_model(hp):
    model = Sequential()
    
    model.add(Dense(32,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [31]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [32]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [36]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7922077775001526

Trial 0 summary
Hyperparameters:
optimizer: adam
Score: 0.7662337422370911

Trial 1 summary
Hyperparameters:
optimizer: sgd
Score: 0.701298713684082

Trial 3 summary
Hyperparameters:
optimizer: adadelta
Score: 0.29870128631591797


In [37]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch = 6,validation_data=(X_test,y_test))

Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7452 - loss: 0.5500 - val_accuracy: 0.7792 - val_loss: 0.5242
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7797 - loss: 0.5210 - val_accuracy: 0.7662 - val_loss: 0.5089
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7668 - loss: 0.5177 - val_accuracy: 0.7662 - val_loss: 0.4987
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7843 - loss: 0.5032 - val_accuracy: 0.7403 - val_loss: 0.4923
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7756 - loss: 0.4937 - val_accuracy: 0.7532 - val_loss: 0.4876
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7859 - loss: 0.5064 - val_accuracy: 0.7532 - val_loss: 0.4851
Epoch 13/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7814 - loss: 0.4811 - val_accuracy: 0.7532 - val_loss: 0.4812
Epoch 14/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7953 - loss: 0.4724 - val_accuracy: 0.753

# 2. No of nodes in a layer

In [55]:
def build_model(hp):
    model = Sequential()
    
    units = hp.Int('Units', 8,128,step=1)
    
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [56]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='my_dir',
                        project_name='Jehan Tuning')

Reloading Tuner from my_dir\Jehan Tuning\tuner0.json


In [57]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [58]:
tuner.results_summary()

Results summary
Results in my_dir\Jehan Tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
Units: 88
Score: 0.7792207598686218

Trial 2 summary
Hyperparameters:
Units: 16
Score: 0.7792207598686218

Trial 1 summary
Hyperparameters:
Units: 72
Score: 0.7662337422370911

Trial 3 summary
Hyperparameters:
Units: 120
Score: 0.7402597665786743

Trial 0 summary
Hyperparameters:
Units: 8
Score: 0.6753246784210205


In [59]:
tuner.get_best_hyperparameters()[0].values

{'Units': 88}

In [60]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [61]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7621 - loss: 0.5206 - val_accuracy: 0.7532 - val_loss: 0.4775
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7897 - loss: 0.4911 - val_accuracy: 0.7532 - val_loss: 0.4655
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7929 - loss: 0.4571 - val_accuracy: 0.7532 - val_loss: 0.4606
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7797 - loss: 0.4789 - val_accuracy: 0.7403 - val_loss: 0.4555
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7767 - loss: 0.4587 - val_accuracy: 0.7532 - val_loss: 0.4521
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7662 - loss: 0.4876 - val_accuracy: 0.7532 - val_loss: 0.4522
Epoch 13/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8077 - loss: 0.4168 - val_accuracy: 0.7532 - val_loss: 0.4508
Epoch 14/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7806 - loss: 0.4624 - val_accuracy: 0.753

# 3. How to select no of hidden layer

In [64]:
def build_model(hp):
    model = Sequential()
    
    model.add(Dense(72,activation='relu',input_dim=8))
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid')) 
    
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [65]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='my_dir',
                        project_name='hidden_layers')

C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 00m 08s


In [67]:
tuner.results_summary()

Results summary
Results in my_dir\hidden_layers
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 7
Score: 0.8311688303947449

Trial 0 summary
Hyperparameters:
num_layers: 1
Score: 0.8051947951316833

Trial 2 summary
Hyperparameters:
num_layers: 10
Score: 0.7922077775001526


In [68]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [69]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [70]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7500 - loss: 0.5068 - val_accuracy: 0.7922 - val_loss: 0.4840
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8012 - loss: 0.4379 - val_accuracy: 0.8052 - val_loss: 0.4473
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8251 - loss: 0.4087 - val_accuracy: 0.7792 - val_loss: 0.4531
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7766 - loss: 0.4424 - val_accuracy: 0.8052 - val_loss: 0.4646
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8091 - loss: 0.4108 - val_accuracy: 0.8052 - val_loss: 0.4273
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7842 - loss: 0.4292 - val_accuracy: 0.7792 - val_loss: 0.4319
Epoch 13/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8311 - loss: 0.3869 - val_accuracy: 0.8052 - val_loss: 0.4131
Epoch 14/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8245 - loss: 0.4001 - val_accuracy: 0.76

# 4. All in all one model 

In [82]:
def build_model(hp):
    
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter == 0:
            model.add(
                Dense(hp.Int('units'+ str(i),min_value=8,max_value=128,step=8),
                      activation = hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),
                      input_dim=8        
                      )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
            
        else:
            model.add(
                Dense(hp.Int('units'+ str(i),min_value=8,max_value=128,step=8),
                      activation = hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid'])
                      )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter +=1    
        
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])   
    
    return model

In [83]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='my_dir',
                        project_name='final')


C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [84]:
tuner.search(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 10s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 31s


In [85]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 128,
 'activation0': 'sigmoid',
 'dropout0': 0.5,
 'optimizer': 'adam',
 'units1': 96,
 'activation1': 'tanh',
 'dropout1': 0.4,
 'units2': 128,
 'activation2': 'relu',
 'dropout2': 0.5,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.3,
 'units4': 104,
 'activation4': 'tanh',
 'dropout4': 0.6,
 'units5': 72,
 'activation5': 'tanh',
 'dropout5': 0.5,
 'units6': 88,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 88,
 'activation7': 'relu',
 'dropout7': 0.6,
 'units8': 88,
 'activation8': 'sigmoid',
 'dropout8': 0.2}

In [86]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [87]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7713 - loss: 0.4857 - val_accuracy: 0.7792 - val_loss: 0.4490
Epoch 7/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7449 - loss: 0.4979 - val_accuracy: 0.7922 - val_loss: 0.4539
Epoch 8/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7589 - loss: 0.4878 - val_accuracy: 0.7792 - val_loss: 0.4465
Epoch 9/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7719 - loss: 0.4959 - val_accuracy: 0.7922 - val_loss: 0.4527
Epoch 10/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7743 - loss: 0.4797 - val_accuracy: 0.7922 - val_loss: 0.4517
Epoch 11/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7718 - loss: 0.4887 - val_accuracy: 0.7922 - val_loss: 0.4509
Epoch 12/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7556 - loss: 0.5035 - val_accuracy: 0.7922 - val_loss: 0.4481
Epoch 13/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7473 - loss: 0.5074 - val_accuracy: 0.7662